<div class='alert alert-warning'>

SciPy's interactive examples with Jupyterlite are experimental and may not always work as expected. Execution of cells containing imports may result in large downloads (up to 60MB of content for the first import from SciPy). Load times when importing from SciPy may take roughly 10-20 seconds. If you notice any problems, feel free to open an [issue](https://github.com/scipy/scipy/issues/new/choose).

</div>

In [ ]:
from scipy.stats.sampling import NumericalInversePolynomial
from scipy.stats import norm
import numpy as np

To create a generator to sample from the standard normal distribution, do:


In [ ]:
class StandardNormal:
   def pdf(self, x):
       return np.exp(-0.5 * x*x)

dist = StandardNormal()
urng = np.random.default_rng()
rng = NumericalInversePolynomial(dist, random_state=urng)

Once a generator is created, samples can be drawn from the distribution by calling
the `rvs` method:


In [ ]:
rng.rvs()

-1.5244996276336318

To check that the random variates closely follow the given distribution, we can
look at it's histogram:


In [ ]:
import matplotlib.pyplot as plt
rvs = rng.rvs(10000)
x = np.linspace(rvs.min()-0.1, rvs.max()+0.1, 1000)
fx = norm.pdf(x)
plt.plot(x, fx, 'r-', lw=2, label='true distribution')
plt.hist(rvs, bins=20, density=True, alpha=0.8, label='random variates')
plt.xlabel('x')
plt.ylabel('PDF(x)')
plt.title('Numerical Inverse Polynomial Samples')
plt.legend()
plt.show()

It is possible to estimate the u-error of the approximated PPF if the exact
CDF is available during setup. To do so, pass a `dist` object with exact CDF of
the distribution during initialization:


In [ ]:
from scipy.special import ndtr
class StandardNormal:
   def pdf(self, x):
       return np.exp(-0.5 * x*x)
   def cdf(self, x):
       return ndtr(x)

dist = StandardNormal()
urng = np.random.default_rng()
rng = NumericalInversePolynomial(dist, random_state=urng)

Now, the u-error can be estimated by calling the `u_error` method. It runs a
Monte-Carlo simulation to estimate the u-error. By default, 100000 samples are
used. To change this, you can pass the number of samples as an argument:


In [ ]:
rng.u_error(sample_size=1000000)  # uses one million samples

UError(max_error=8.785994154436594e-11, mean_absolute_error=2.930890027826552e-11)

This returns a namedtuple which contains the maximum u-error and the mean
absolute u-error.

The u-error can be reduced by decreasing the u-resolution (maximum allowed u-error):


In [ ]:
urng = np.random.default_rng()
rng = NumericalInversePolynomial(dist, u_resolution=1.e-12, random_state=urng)
rng.u_error(sample_size=1000000)

UError(max_error=9.07496300328603e-13, mean_absolute_error=3.5255644517257716e-13)

Note that this comes at the cost of increased setup time.

The approximated PPF can be evaluated by calling the `ppf` method:


In [ ]:
rng.ppf(0.975)

1.9599639857012559

In [ ]:
norm.ppf(0.975)

1.959963984540054

Since the PPF of the normal distribution is available as a special function, we
can also check the x-error, i.e. the difference between the approximated PPF and
exact PPF
```

```

In [ ]:
import matplotlib.pyplot as plt
u = np.linspace(0.01, 0.99, 1000)
approxppf = rng.ppf(u)
exactppf = norm.ppf(u)
error = np.abs(exactppf - approxppf)
plt.plot(u, error)
plt.xlabel('u')
plt.ylabel('error')
plt.title('Error between exact and approximated PPF (x-error)')
plt.show()